In [5]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import datetime
import generate_oil_analysis

# Set today's date for analysis
today = pd.Timestamp("2025-08-28")

# Load price data from user-provided CSV (Brent, RBOB, ULSD)
# csv_path = Path('/home/oai/share/data/oil_prices.csv')
csv_path = Path('data/oil_prices.csv')  # use repo-local data folder
prices = generate_oil_analysis.load_price_history(csv_path, today)

# Compute the crack spread and simple refinery margin proxy
prices = generate_oil_analysis.compute_crack(prices)

# Display today's crack value and its percentile within the 3-year history
today_crack = prices.loc[prices['date'] == prices['date'].max(), 'crack'].iloc[0]
percentile = generate_oil_analysis.compute_percentile(prices['crack'], today_crack)
print(f"Today's 3‑2‑1 crack: {today_crack:.1f} USD/bbl (percentile: {percentile:.1f}%)")

# output paths — use repo-local outputs directory
out_dir = Path("outputs")
out_dir.mkdir(parents=True, exist_ok=True)

output_png = out_dir / "oil_crack.png"
generate_oil_analysis.plot_crack_time_series(prices, today, output_png)

# Build a scenario table for ±$1/bbl crude moves and ±$0.05/gal product moves
latest = prices.iloc[-1]
scenario_df = generate_oil_analysis.build_scenario_table(latest['crude'], latest['rbob'], latest['ulsd'])
# Save scenarios to CSV (repo-local)
scenario_df.to_csv(out_dir / "oil_scenarios.csv", index=False)

# Display scenario table
scenario_df

# Write market note and fundamentals PDF using cited references from EIA
citation_refs = ('168506629185033†L8-L16', '168506629185033†L18-L23')
note_path = out_dir / "oil_note.md"
pdf_path = out_dir / "fundamentals.pdf"

generate_oil_analysis.write_market_note(prices, today, note_path, citation_refs)
generate_oil_analysis.write_fundamentals_pdf(pdf_path, citation_refs)

print(f"Outputs saved to {out_dir}: oil_crack.png, oil_scenarios.csv, oil_note.md, fundamentals.pdf")


Today's 3‑2‑1 crack: 29.8 USD/bbl (percentile: 15.8%)
Outputs saved to outputs: oil_crack.png, oil_scenarios.csv, oil_note.md, fundamentals.pdf
